In [1]:
# STD
import codecs
from functools import reduce
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import time
import math
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [2]:
batch_size = 64
embedding_dims = 100
hidden_dims = embedding_dims*2
en_file = "data/train/train.en"
fr_file = "data/train/train.fr"
num_epochs = 30
learning_rate = 0.01
max_sentence_len = 50
cuda = True

# To Do

    -Currently has 17k vocab size for french and 15k for english, smaller? seems fast enough... not neccesairy?
    
    
# usefulll for later

    -That is correct. Simply set ignore_index to your padding index. Also set size_average to false and divide the loss by the number of non padding tokens. <<--- someone said this, but cant you just let loss itself avarage it out by not setting it to false? thats what i did now, might try some runs with this method when we have an evaluation method to check for difference.
    
# questions


In [3]:
# same code as from model.py
# copied here so i could make adjustments as i see fit

# if code is commented out its old code that is no longer used

# >>>>>>
# is new code
# <<<<<<

class ParallelCorpus(Dataset):
    """
    Class that contains a parallel corpus used for training IBM Model 1 and 2.
    """
   
    #  def __init__(self, source_path, target_path):
    
    # >>>>>>>>>>>>>> added additional inputs.
    def __init__(self, source_path, target_path, max_sentence_len, cuda=False):
    # <<<<<<<<<<<<<<
    
        # Enable a way to read multiple paths into one corpus if wanted
        source_paths = source_path if self.is_listlike(source_path) else (source_path, )
        target_paths = target_path if self.is_listlike(target_path) else (target_path, )

        self.source_sentences = self.read_all(source_paths)
        self.target_sentences = self.read_all(target_paths)
        
        # >>>>> Limit to sentences shorter then max_sentence_size
        # also adds SOS and EOS tags to target sentence
        self.source_sentences = [self.source_sentences[i] for i in range(len(self.source_sentences)) if len(self.target_sentences[i]) <= max_sentence_len]
        self.target_sentences = [["<SOS>"] + sentence + ["<EOS>"] for sentence in self.target_sentences if len(sentence) <= max_sentence_len]
        # <<<<<
        
        self.size = len(self.source_sentences)
        
        # self.source_vocab = set()
        # self._source_vocab_size = None
        
        
    # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 
    
        # getting additional vocab info
        self.source_vocab, self.source_w2i, self.source_i2w, self.source_vocab_size = self.set_vocab(self.source_sentences)  
        self.target_vocab, self.target_w2i, self.target_i2w, self.target_vocab_size = self.set_vocab(self.target_sentences)
        
        self.source_vocab_size = len(self.source_vocab)
        self.target_vocab_size = len(self.target_vocab)
        
        # convert word sentences to index sentences
        self.source_idx = self.set_index_sentences(self.source_sentences, self.source_w2i)
        self.target_idx = self.set_index_sentences(self.target_sentences, self.target_w2i)
        
        # getting the lengths of the sentences
        self.source_lengths = [len(s) for s in self.source_idx]
        self.target_lengths = [len(s) for s in self.target_idx]
        
        # create postition vectors
        self.pos_base = [i for i in range(1,max_sentence_len+1)]
        self.positions = [self.pos_base[:i] for i in self.source_lengths]
        
        self.source_pad = self.source_w2i['<pad>']
        self.target_pad = self.target_w2i['<pad>']
        
        # pad the sentences
        self.source_padded = self.pad_sequences(self.source_idx, self.source_lengths, self.source_pad)
        self.target_padded = self.pad_sequences(self.target_idx, self.target_lengths, self.target_pad)
        self.positions = self.pad_sequences(self.positions, self.source_lengths, 0)
        
        # turn into tensors
        if (cuda):
            self.source_tensor = torch.LongTensor(self.source_padded).cuda()
            self.target_tensor = torch.LongTensor(self.target_padded).cuda()
            self.source_lengths = torch.LongTensor(self.source_lengths).cuda()
            self.target_lengths = torch.LongTensor(self.target_lengths).cuda()
            self.positions = torch.LongTensor(self.positions).cuda()
        else:
            self.source_tensor = torch.LongTensor(self.source_padded)
            self.target_tensor = torch.LongTensor(self.target_padded)
            self.source_lengths = torch.LongTensor(self.source_lengths)
            self.target_lengths = torch.LongTensor(self.target_lengths)
            self.positions = torch.LongTensor(self.positions)
            
        # sort them by sentence size, decending.
        self.source_tensor, self.target_tensor, self.source_lengths, self.target_lengths, self.positions = \
            self.sort_data(self.source_tensor, self.target_tensor, self.source_lengths, self.target_lengths, self.positions)
        
    def set_vocab(self, sentences):
        vocab = ['<pad>', '<unk>', "<SOS>", "<EOS>"]
        for line in sentences:
            for word in line:
                if word not in vocab:
                    vocab.append(word)
                    
        word2idx = {w: idx for (idx, w) in enumerate(vocab)}
        idx2word = {idx: w for (idx, w) in enumerate(vocab)}

        vocab_size = len(vocab)
        
        return vocab, word2idx, idx2word, vocab_size
        
    def set_index_sentences(self, sentences, w2i):
        sentence_indexs = []
        for sentence in sentences:
            sentence_indexs.append([w2i[word] for word in sentence])
        return sentence_indexs
    
    def pad_sequences(self, sentence_indexs, seq_lengths, pad):
        padding = np.full((len(sentence_indexs),max(seq_lengths)),pad) 
        for idx, (seq, seqlen) in enumerate(zip(sentence_indexs, seq_lengths)):
            padding[idx, :seqlen] = seq
        return padding
    
    def sort_data(self, source, target, source_lengths ,target_lengths, pos):

        target_lengths, perm_idx = target_lengths.sort(0, descending=True)
        source_tensor = source[perm_idx]
        target_tensor = target[perm_idx]
        source_lengths = source_lengths[perm_idx]
        positions = pos[perm_idx]
        return source_tensor, target_tensor, source_lengths, target_lengths, positions
    
    # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

        

    def read_all(self, paths):
        """
        Read all corpus file at given paths and merge results.
        """
        def _combine_lists(a, b):
            a.extend(b)
            return a

        sentences_per_path = [self.read_corpus_file(path) for path in paths]
        return reduce(_combine_lists, sentences_per_path)

    @staticmethod
    def read_corpus_file(path, filter_characters=[]):
        sentences = []

        with codecs.open(path, "rb", "utf-8") as corpus:
            for line in corpus.readlines():
                tokens = [token for token in line.strip().split() if token not in filter_characters]
                sentences.append(tokens)

        return sentences

#     @property
#     def parallel_sentences(self):
#         return zip(self.source_sentences, self.target_sentences)

#     @property
#     def source_vocab_size(self):
#         if self._source_vocab_size is None:
#             self.source_vocab = {token for sentence in self.target_sentences for token in sentence}
#             self._source_vocab_size = len(self.source_vocab)
#         return self._source_vocab_size

#     def __iter__(self):
#         return (
#             (source_sentence, target_sentence)
#             for source_sentence, target_sentence in zip(self.source_sentences, self.target_sentences)
#         )

#     def __len__(self):
#         return self.size

#     def __getitem__(self, item):
#         return self.source_sentences[item], self.target_sentences[item]


    # >>>>>>>>>>>>>
    def __len__(self):
        return self.source_tensor.size(0) 

    def __getitem__(self, index):
        return self.source_tensor[index], self.target_tensor[index], self.source_lengths[index], self.target_lengths[index], self.positions[index]
    # <<<<<<<<<<<<<<<
    
    
    @staticmethod
    def is_listlike(obj):
        return type(obj) in (tuple, list, set)


In [4]:
dataset = ParallelCorpus(fr_file, en_file, max_sentence_len, cuda)

In [5]:
data_loader = DataLoader(dataset, batch_size=batch_size)

In [6]:
class Encoder(nn.Module):
    def __init__(self, source_vocab_size, embedding_dims,max_sentence_len):
        super(Encoder, self).__init__()
        self.word_embeddings = nn.Embedding(source_vocab_size, embedding_dims) #embed words
        self.pos_embeddings = nn.Embedding(max_sentence_len+1, embedding_dims) #embed positions
        
    def forward(self, source_sentences, positions):
        words = self.word_embeddings(source_sentences)
        pos = self.pos_embeddings(positions)
        return torch.cat((words,pos),2)
        

In [7]:
class Decoder(nn.Module):
    def __init__(self, target_vocab_size, embedding_dims):
        super(Decoder, self).__init__()
        
        # Decoder stuff
        self.word_embeddings = nn.Embedding(target_vocab_size, embedding_dims)
        self.lstm = nn.LSTM(embedding_dims, hidden_dims, batch_first=True)
        self.scale_h0 = nn.Linear(embedding_dims*2, hidden_dims)
        self.out = nn.Linear(embedding_dims*2 + hidden_dims, target_vocab_size)
        
        #Attention stuff
        
        # used only in the concat version
        self.attn_layer = nn.Linear(embedding_dims*2 + hidden_dims, 1)
        
        self.attn_soft = nn.Softmax(dim=2)
        
    def forward(self, target_sentences, target_lengths, source_lengths, encoder_outputs, max_len):
        words = self.word_embeddings(target_sentences)
        
        # avg out encoder data to use as hidden state
        avg = torch.mean(encoder_outputs,1)
        hidden = self.scale_h0(avg)
        
        # het hidden state from encoder RNN
        packed_input = pack_padded_sequence(words, target_lengths, batch_first=True)
        packed_output, (ht, ct) = self.lstm(packed_input, (torch.unsqueeze(hidden,0),torch.unsqueeze(hidden,0)))
        lstm_out, _ = pad_packed_sequence(packed_output, batch_first=True)
        
        # prepare lstm Hidden layers for input into attention,
        # we add the hidden layer as zeroth lstm_out and remove the last one
        # this is because we need h_i-1, not h_i
        lstm_to_attn = torch.cat((torch.unsqueeze(hidden,1),lstm_out),1)
        lstm_to_attn = lstm_to_attn = lstm_to_attn[:,:-1,:]
        
        context = self.attention(lstm_to_attn, encoder_outputs, source_lengths, max_len)
        
        
        # combine with non existing context vectors
        combined = torch.cat((lstm_out,lstm_out),2)
        out = self.out(combined)
        return out
    
    def attention(self, lstm_to_attn, encoder_outputs, source_lengths, max_len):
                
        # repeat the lstm out in third dimension and the encoder outputs in second dimension so we can make a meshgrid
        # so we can do elementwise mul for all possible combinations of h_j and s_i
        h_j = encoder_outputs.unsqueeze(1).repeat(1,lstm_to_attn.size(1),1,1)
        s_i = lstm_to_attn.unsqueeze(2).repeat(1,1,encoder_outputs.size(1),1)
        
        # get the dot product between the two to get the energy
        # the unsqueezes are there to emulate transposing. so we can use matmul as torch.dot doesnt accept matrices
        energy = s_i.unsqueeze(3).matmul(h_j.unsqueeze(4)).squeeze(4)
        
#         # this is concat attention, its a different form then the ones we need
#         cat = torch.cat((s_i,h_j),3)
        
#         energy = self.attn_layer(cat)

        # reshaping the encoder outputs for later
        encoder_outputs = encoder_outputs.unsqueeze(1)
        encoder_outputs = encoder_outputs.repeat(1,energy.size(1),1,1)
    
        # apply softmax to the energys 
        allignment = self.attn_soft(energy)
        
        # create a mask like : [1,1,1,0,0,0] whos goal is to multiply the attentions of the pads with 0, rest with 1
        idxes = torch.arange(0,max_len,out=max_len).unsqueeze(0)
        mask = Variable((idxes<source_lengths.unsqueeze(1)).float())
        
        # format the mask to be same size() as the attentions
        mask = mask.unsqueeze(1).unsqueeze(3).repeat(1,allignment.size(1),1,1)
        
        # apply mask
        masked = allignment * mask
        
        # now we have to rebalance the other values so they sum to 1 again
        # this is done by dividing each value by the sum of the sequence
        # calculate sums
        msum = masked.sum(-2).repeat(1,1,masked.size(2)).unsqueeze(3)
        
        # rebalance
        attentions = masked.div(msum)
        
        # now we shape the attentions to be similar to context in size
        allignment = allignment.repeat(1,1,1,encoder_outputs.size(3))

        # make context vector by element wise mul
        context = attentions * encoder_outputs
        
        return context
        
        

In [8]:
# initialiase models, loss and optimizer
if (cuda):
    encoder = Encoder(dataset.source_vocab_size, embedding_dims, max_sentence_len).cuda()
    decoder = Decoder(dataset.target_vocab_size, embedding_dims).cuda()
    loss_function = nn.CrossEntropyLoss(ignore_index = dataset.target_pad).cuda()
else:
    encoder = Encoder(dataset.source_vocab_size, embedding_dims, max_sentence_len).cpu()
    decoder = Decoder(dataset.target_vocab_size, embedding_dims).cpu()
    loss_function = nn.CrossEntropyLoss(ignore_index = dataset.target_pad).cpu()
    
optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), learning_rate)

In [9]:
epoch_losses = []
iterations = len(data_loader)

for epoch in range(0, num_epochs):
    start = time.time()
    batch_losses = []
    batch = 0
    for source_batch, target_batch, source_lengths, target_lengths, batch_positions in data_loader:
        batch_start = time.time()
        max_len = source_lengths.max()
        source_batch = source_batch[:,:max_len]
        batch_positions = batch_positions[:,:max_len]
        
        source_batch = torch.autograd.Variable(source_batch)
        target_batch = torch.autograd.Variable(target_batch)
        source_lengths = torch.autograd.Variable(source_lengths)
        target_lengths = torch.autograd.Variable(target_lengths)
        batch_positions = torch.autograd.Variable(batch_positions)
        
        if (cuda):
            source_batch = source_batch.cuda()
            target_batch = target_batch.cuda()
            source_lengths = source_lengths.cuda()
            target_lengths = target_lengths.cuda()
            batch_positions = batch_positions.cuda()
            
            
        # get encoder and decoder outputs
        encoder_out = encoder(source_batch, batch_positions)
        decoder_out = decoder(target_batch, target_lengths, source_lengths, encoder_out, max_len)
        
        # remove the start token from the targets and the end token from the decoder
        decoder_out2 = decoder_out[:,:-1,:]
        target_batch2 = target_batch[:,1:decoder_out.size(1)]
        
        # pytorch expects the inputs for the loss function to be: (batch x classes)
        # cant handle sentences so we just transform our data to treath each individual word as a batch:
        # so new batch size = old batch * padded sentence length
        # should not matter as it all gets averaged out anyway
        decoder_out3 = decoder_out2.contiguous().view(decoder_out2.size(0)*decoder_out2.size(1),decoder_out2.size(2))
        target_batch3 = target_batch2.contiguous().view(target_batch2.size(0)*target_batch2.size(1))
        
        # calculate loss
        loss = loss_function(decoder_out3,target_batch3)
        batch_losses.append(loss)
        
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_time = time.time() - batch_start 
        print('\r[Epoch {:03d}/{:03d}] Batch {:06d}/{:06d} [{:.1f}/s] '.format(epoch+1, num_epochs, batch+1, iterations, batch_time), end='')
        batch += 1
        
    avg_loss = sum(batch_losses) / iterations 
    epoch_losses.append(avg_loss)
    print('Time: {:.1f}s Loss: {:.3f} score2?: {:.6f}'.format(time.time() - start, avg_loss, 0))

        
        
        
        
        

[Epoch 001/030] Batch 000454/000454 [0.0/s] Time: 36.7s Loss: 3.968 score2?: 0.000000
[Epoch 002/030] Batch 000454/000454 [0.0/s] Time: 36.2s Loss: 3.176 score2?: 0.000000
[Epoch 003/030] Batch 000454/000454 [0.0/s] Time: 36.2s Loss: 2.605 score2?: 0.000000
[Epoch 004/030] Batch 000454/000454 [0.0/s] Time: 36.1s Loss: 2.218 score2?: 0.000000
[Epoch 005/030] Batch 000454/000454 [0.0/s] Time: 36.1s Loss: 1.938 score2?: 0.000000
[Epoch 006/030] Batch 000454/000454 [0.0/s] Time: 36.1s Loss: 1.740 score2?: 0.000000
[Epoch 007/030] Batch 000454/000454 [0.1/s] Time: 36.2s Loss: 1.572 score2?: 0.000000
[Epoch 008/030] Batch 000454/000454 [0.0/s] Time: 36.2s Loss: 1.444 score2?: 0.000000
[Epoch 009/030] Batch 000454/000454 [0.0/s] Time: 36.1s Loss: 1.351 score2?: 0.000000
[Epoch 010/030] Batch 000454/000454 [0.0/s] Time: 36.1s Loss: 1.272 score2?: 0.000000
[Epoch 011/030] Batch 000454/000454 [0.0/s] Time: 36.1s Loss: 1.201 score2?: 0.000000
[Epoch 012/030] Batch 000454/000454 [0.1/s] Time: 36.2

# Everything below is non usefull, testing code used to try stuff